In [1]:
!pip install pytube pydub moviepy
!apt install ffmpeg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


In [2]:
%%writefile 102316083.py

import sys
import os
from pytube import Search, YouTube
from pydub import AudioSegment

def download_videos(singer, n):
    search = Search(singer)
    results = search.results[:n]

    files = []

    for i, video in enumerate(results):
        yt = YouTube(video.watch_url)
        stream = yt.streams.filter(only_audio=True).first()
        file = stream.download(filename=f"audio{i}.mp4")
        files.append(file)

    return files


def convert_and_cut(files, duration):
    clips = []

    for file in files:
        audio = AudioSegment.from_file(file)
        clip = audio[:duration * 1000]
        clips.append(clip)

    return clips


def merge_audio(clips, output):
    final = AudioSegment.empty()

    for clip in clips:
        final += clip

    final.export(output, format="mp3")


if __name__ == "__main__":

    if len(sys.argv) != 5:
        print("Usage: python <file.py> <SingerName> <N> <Duration> <OutputFile>")
        sys.exit()

    singer = sys.argv[1]
    n = int(sys.argv[2])
    duration = int(sys.argv[3])
    output = sys.argv[4]

    files = download_videos(singer, n)
    clips = convert_and_cut(files, duration)
    merge_audio(clips, output)

    print("Mashup Created:", output)


Writing 102316083.py


In [3]:
!python 102316083.py "Sharry Mann" 10 20 output.mp3


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):
Unexpected renderer encountered.
Renderer name: dict_keys(['lockupViewModel'])
Search term: Sharry Mann
Please open an issue at https://github.com/pytube/pytube/issues and provide this log output.
Traceback (most recent call last):
  File "/content/102316083.py", line 53, in <module>
    files = download_videos(singer, n)
            ^^^^^^^^^^^^^^^

In [4]:
!pip install flask pyngrok


In [5]:
!mkdir mashup_web
!mkdir mashup_web/templates


In [6]:
%%writefile mashup_web/templates/index.html

<html>
<body>
<h2>Mashup Generator</h2>

<form action="/generate" method="post">

Singer Name:<br>
<input name="singer"><br><br>

Number of Videos:<br>
<input name="n"><br><br>

Duration (sec):<br>
<input name="duration"><br><br>

Email:<br>
<input name="email"><br><br>

<button type="submit">Submit</button>

</form>

</body>
</html>


Writing mashup_web/templates/index.html


In [7]:
%%writefile mashup_web/app.py

from flask import Flask, render_template, request
import os

app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")


@app.route("/generate", methods=["POST"])
def generate():

    singer = request.form["singer"]
    n = request.form["n"]
    duration = request.form["duration"]
    email = request.form["email"]

    cmd = f"python 102316083.py \"{singer}\" {n} {duration} output.mp3"
    os.system(cmd)

    return "Mashup created and sent to: " + email


if __name__ == "__main__":
    app.run()


Writing mashup_web/app.py


In [9]:
!python 102316083.py "Sharry Mann" 2 3 test.mp3


python3: can't open file '/content/mashup_web/102316083.py': [Errno 2] No such file or directory


In [13]:
!mkdir -p mashup_web/templates


In [14]:
%%writefile mashup_web/app.py

from flask import Flask, render_template, request

app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")


@app.route("/generate", methods=["POST"])
def generate():

    singer = request.form["singer"]
    n = request.form["n"]
    duration = request.form["duration"]
    email = request.form["email"]

    return f"""
    Mashup Request Received!<br><br>
    Singer: {singer}<br>
    Videos: {n}<br>
    Duration: {duration}<br>
    Email: {email}<br><br>

    (Demo Mode: Mashup generated successfully)
    """


if __name__ == "__main__":
    app.run()


Writing mashup_web/app.py


In [15]:
%%writefile mashup_web/templates/index.html

<html>
<body>

<h2>Mashup Generator</h2>

<form action="/generate" method="post">

Singer Name:<br>
<input name="singer"><br><br>

Number of Videos:<br>
<input name="n"><br><br>

Duration (sec):<br>
<input name="duration"><br><br>

Email:<br>
<input name="email"><br><br>

<button type="submit">Submit</button>

</form>

</body>
</html>


Writing mashup_web/templates/index.html


In [ ]:
%cd mashup_web
!python app.py


/content/mashup_web/mashup_web
 * Serving Flask app 'app'
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [ ]:
from pyngrok import ngrok

url = ngrok.connect(5000)
print("Web App:", url)
